In [ ]:
# Import standard modules.
from importlib import import_module
import math as m
import os
import sys

# Import supplemental modules.
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import numpy as np
import seaborn as sb

# Import project modules.
import pinn.standard_plots as psp

In [ ]:
# Specify the run ID (aka problem name).
runid = "linecurrent_nPuxBxByBz"

# Add the subdirectory for the run results to the module search path.
run_path = os.path.join(".", runid)
sys.path.append(run_path)

# Import the problem definition from the run results directory.
p = import_module(runid)

# Read the run hyperparameters from the run results directory.
import hyperparameters as hp

In [ ]:
# Define analytical solutions.

# Constants
mu0 = 1.0  # Normalized vacuum permittivity.
I = 1e-3    # Normalized current.
Q = 60.0   # Flow angle in degrees clockwise from +y axis.
u0 = 1.0   # Initial flow speed.

# Compute the constant velocity components.
u0x = u0*np.sin(np.radians(Q))
u0y = u0*np.cos(np.radians(Q))

# Constant values for other variables.
n0 = 1.0
P0 = 1.0
B0z = 0.0


def n_analytical(t, x, y):
    n = np.full(t.shape, n0)
    return n

def P_analytical(t, x, y):
    P = np.full(t.shape, P0)
    return P

def ux_analytical(t, x, y):
    ux = np.full(t.shape, u0x)
    return ux

def Bx_analytical(t, x, y):
    r = np.sqrt((x - u0x*t)**2 + (y - u0y*t)**2)
    Bx = -mu0*I/(2*np.pi)*(y - u0y*t)/r**2
    return Bx

def By_analytical(t, x, y):
    r = np.sqrt((x - u0x*t)**2 + (y - u0y*t)**2)
    By = mu0*I/(2*np.pi)*(x - u0x*t)/r**2
    return By

def Bz_analytical(t, x, y):
    Bz = np.full(t.shape, B0z)
    return Bz

def dBx_dx_analytical(t, x, y):
    r = np.sqrt((x - u0x*t)**2 + (y - u0y*t)**2)
    dBx_dx = mu0*I/np.pi * (x - u0x*t)*(y - u0y*t)/r**4
    return dBx_dx

def dBy_dy_analytical(t, x, y):
    r = np.sqrt((x - u0x*t)**2 + (y - u0y*t)**2)
    dBy_dy = -mu0*I/np.pi * (x - u0x*t)*(y - u0y*t)/r**4
    return dBy_dy

In [ ]:
# Load all data.

# Load the training point coordinates.
txy_train = np.loadtxt(os.path.join(runid, "X_train.dat"))
t_train = txy_train[:, 0]
x_train = txy_train[:, 1]
y_train = txy_train[:, 2]

# Load the data locations and values (includes initial conditions).
txy_data = np.loadtxt(os.path.join(runid, "XY_data.dat"))

# Extract the initial conditions.
ic = txy_data[:, 3:]

# Load the model-predicted values.
ψ = []
delψ = []
for i in range(len(p.dependent_variable_names)):
    var_name = p.dependent_variable_names[i]
    ψ.append(np.loadtxt(os.path.join(runid, "%s_train.dat" % var_name)))
    delψ.append(np.loadtxt(os.path.join(runid, "del_%s_train.dat" % var_name)))

# Load the loss function histories.
losses_model = np.loadtxt(os.path.join(runid, "losses_model.dat"))
losses_model_res = np.loadtxt(os.path.join(runid, "losses_model_res.dat"))
losses_model_data = np.loadtxt(os.path.join(runid, "losses_model_data.dat"))
losses = np.loadtxt(os.path.join(runid, "losses.dat"))
losses_res = np.loadtxt(os.path.join(runid, "losses_res.dat"))
losses_data = np.loadtxt(os.path.join(runid, "losses_data.dat"))

In [ ]:
# Compute the limits of the training domain.
t_min = t_train[0]
t_max = t_train[-1]
x_min = x_train[0]
x_max = x_train[-1]
y_min = y_train[0]
y_max = y_train[-1]

# Extract the unique training point values (a grid is assumed).
t_train_vals = np.unique(t_train)
x_train_vals = np.unique(x_train)
y_train_vals = np.unique(y_train)
n_t_train_vals = len(t_train_vals)
n_x_train_vals = len(x_train_vals)
n_y_train_vals = len(y_train_vals)

In [ ]:
# Plotting options

# Compute the number of rows and columns for the 2-per-row plots.
# N_COLS = 2
# n_rows = m.ceil(p.n_var/N_COLS)

# Specify the size (inches) for individual subplots.
SUBPLOT_WIDTH = 5.0
SUBPLOT_HEIGHT = 5.0

# Compute the figure size for model loss plots.
# model_loss_figsize = (SUBPLOT_WIDTH*N_COLS, SUBPLOT_HEIGHT*n_rows)

# Compute the figure size for the total loss plot.
total_loss_figsize = (SUBPLOT_WIDTH*2, SUBPLOT_HEIGHT)

# Compute the figure size for side-by-side actual and predicted quiver plots for the magnetic field vectors.
# B_vector_figsize = (SUBPLOT_WIDTH*2, SUBPLOT_HEIGHT)

# Compute the figure size for side-by-side actual, predicted, and error plots for the magnetic field magnitudes.
# B_magnitude_figsize = (SUBPLOT_WIDTH*3, SUBPLOT_HEIGHT)

# Magnetic field intensity limits.
# B_VMIN = 1e-4
# B_VMAX = 1

# Density limits.
# N_VMIN = 1 - 1e-6
# N_VMAX = 1 + 1e-6

# Pressure limits.
# P_VMIN = 1 - 1e-6
# P_VMAX = 1 + 1e-6

# Velocity limits
# U_VMIN = 0
# U_VMAX = 1

# Bz limits.
# BZ_VMIN = -1e-6
# BZ_VMAX = 1e-6

# Absolute error heatmap limits.
# ABS_ERR_VMIN = -1e-4
# ABS_ERR_VMAX = 1e-4

# Absolute error in B heatmap limits.
# B_ABS_ERR_VMIN = -1e-3
# B_ABS_ERR_VMAX = 1e-3

# Plot limits for div B heatmap.
# DIV_B_VMIN = -1e-3
# DIV_B_VMAX = 1e-3

# Compute the coordinate plot tick locations and labels.
XY_N_X_TICKS = 5
XY_x_tick_pos = np.linspace(x_min, x_max, XY_N_X_TICKS)
XY_x_tick_labels = ["%.1f" % x for x in XY_x_tick_pos]
XY_N_Y_TICKS = 5
XY_y_tick_pos = np.linspace(y_min, y_max, XY_N_Y_TICKS)
XY_y_tick_labels = ["%.1f" % y for y in XY_y_tick_pos]

# Compute the heat map tick locations and labels.
HEATMAP_N_X_TICKS = 5
heatmap_x_tick_pos = np.linspace(0, n_x_train_vals - 1, HEATMAP_N_X_TICKS)
heatmap_x_tick_labels = ["%.1f" % (x_min + x/(n_x_train_vals - 1)*(x_max - x_min)) for x in heatmap_x_tick_pos]
HEATMAP_N_Y_TICKS = 5
heatmap_y_tick_pos = np.linspace(0, n_y_train_vals - 1, HEATMAP_N_Y_TICKS)
heatmap_y_tick_labels = ["%.1f" % (y_min + y/(n_y_train_vals - 1)*(y_max - y_min)) for y in heatmap_y_tick_pos]
heatmap_y_tick_labels = list(reversed(heatmap_y_tick_labels))

In [ ]:
# Plot the loss history for each model.
fig = psp.plot_model_loss_functions(losses_model_res, losses_model_data, losses_model, p.dependent_variable_labels)

In [ ]:
# Plot the total loss function history.
plt.figure(figsize=total_loss_figsize)
psp.plot_loss_functions(
    [losses_res, losses_data, losses],
    ["$L_{res}$", "$L_{data}$", "$L$"],
    title="Total loss function evolution for %s" % runid
)

In [ ]:
# Plot the actual and predicted initial magnetic field vectors.
n_start = n_x_train_vals*n_y_train_vals
t = txy_data[:, 0]
x = txy_data[:, 1]
y = txy_data[:, 2]
B0x_act = Bx_analytical(t, x, y)
B0y_act = By_analytical(t, x, y)
B0x_pred = ψ[3][:n_start]
B0y_pred = ψ[4][:n_start]

# Create the figure.
fig = psp.plot_actual_predicted_B(
    x, y, B0x_act, B0y_act, B0x_pred, B0y_pred,
    title="Initial magnetic field",
    x_tick_pos=XY_x_tick_pos, x_tick_labels=XY_x_tick_labels,
    y_tick_pos=XY_y_tick_pos, y_tick_labels=XY_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute errors in initial magnetic field magnitudes.
B0_act = np.sqrt(B0x_act**2 + B0y_act**2)
B0_pred = np.sqrt(B0x_pred**2 + B0y_pred**2)
B0_err = B0_pred - B0_act

# To get the proper orientation, reshape, transpose, flip.
B0_act_plot = np.flip(B0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B0_pred_plot = np.flip(B0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B0_err_plot = np.flip(B0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the plot.
fig = psp.plot_actual_predicted_error_Bmag(
    x, y, B0_act_plot, B0_pred_plot, B0_err_plot,
    title="Initial magnetic field magnitude",
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and errors in initial magnetic field divergence.
dB0x_dx_act = dBx_dx_analytical(t, x, y)
dB0y_dy_act = dBy_dy_analytical(t, x, y)
divB0_act = dB0x_dx_act + dB0y_dy_act
dB0x_dx_pred = delψ[3][:n_start, 1]
dB0y_dy_pred = delψ[4][:n_start, 2]
divB0_pred = dB0x_dx_pred + dB0y_dy_pred
divB0_err = divB0_pred - divB0_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
divB0_act_plot = np.flip(divB0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    divB0_act_plot,
    ax, title="Actual",
    vmin=DIV_B_VMIN, vmax=DIV_B_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
divB0_pred_plot = np.flip(divB0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    divB0_pred_plot,
    ax, title="Predicted",
    vmin=DIV_B_VMIN, vmax=DIV_B_VMAX,
    show_ylabel=False,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
divB0_err_plot = np.flip(divB0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    divB0_err_plot,
    ax, title="Error",
    vmin=DIV_B_VMIN, vmax=DIV_B_VMAX,
    show_ylabel=False,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Initial magnetic field divergence")
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in initial number density.
n0_act = n_analytical(t, x, y)
n0_pred = ψ[0][:n_start]
n0_err = n0_pred - n0_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
n0_act_plot = np.flip(n0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    n0_act_plot,
    ax, title="Actual",
    vmin=N_VMIN, vmax=N_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
n0_pred_plot = np.flip(n0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    n0_pred_plot,
    ax, title="Predicted",
    vmin=N_VMIN, vmax=N_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
n0_err_plot = np.flip(n0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    n0_err_plot,
    ax, title="Absolute error",
    vmin=ABS_ERR_VMIN, vmax=ABS_ERR_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Initial %s" % p.dependent_variable_labels[0])
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in initial pressure.
P0_act = P_analytical(t, x, y)
P0_pred = ψ[1][:n_start]
P0_err = P0_pred - P0_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
P0_act_plot = np.flip(P0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    P0_act_plot,
    ax, title="Actual",
    vmin=P_VMIN, vmax=P_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
P0_pred_plot = np.flip(P0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    P0_pred_plot,
    ax, title="Predicted",
    vmin=P_VMIN, vmax=P_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
P0_err_plot = np.flip(P0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    P0_err_plot,
    ax, title="Absolute error",
    vmin=ABS_ERR_VMIN, vmax=ABS_ERR_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Initial %s" % p.dependent_variable_labels[1])
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in x-velocity.
u0x_act = ux_analytical(t, x, y)
u0x_pred = ψ[2][:n_start]
u0x_err = u0x_pred - u0x_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
u0x_act_plot = np.flip(u0x_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    u0x_act_plot,
    ax, title="Actual",
    vmin=U_VMIN, vmax=U_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
u0x_pred_plot = np.flip(u0x_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    u0x_pred_plot,
    ax, title="Predicted",
    vmin=U_VMIN, vmax=U_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
u0x_err_plot = np.flip(u0x_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    u0x_err_plot,
    ax, title="Absolute error",
    vmin=ABS_ERR_VMIN, vmax=ABS_ERR_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Initial %s" % p.dependent_variable_labels[2])
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in initial Bz.
B0z_act = Bz_analytical(t, x, y)
B0z_pred = ψ[5][:n_start]
B0z_err = B0z_pred - B0z_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
B0z_act_plot = np.flip(B0z_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    B0z_act_plot,
    ax, title="Actual",
    vmin=BZ_VMIN, vmax=BZ_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
B0z_pred_plot = np.flip(B0z_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    B0z_pred_plot,
    ax, title="Predicted",
    vmin=BZ_VMIN, vmax=BZ_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
B0z_err_plot = np.flip(B0z_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    B0z_err_plot,
    ax, title="Absolute error",
    vmin=B_ABS_ERR_VMIN, vmax=B_ABS_ERR_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Initial %s" % p.dependent_variable_labels[5])
plt.show()

In [ ]:
# Plot the actual and predicted final magnetic field vectors.
n_end = n_x_train_vals*n_y_train_vals
t = t_train[-n_end:]
x = x_train[-n_end:]
y = y_train[-n_end:]
B1x_act = Bx_analytical(t, x, y)
B1y_act = By_analytical(t, x, y)
B1x_pred = ψ[3][-n_end:]
B1y_pred = ψ[4][-n_end:]

# Create the figure.
fig = plt.figure(figsize=B_vector_figsize)

# Actual
ax = plt.subplot(1, 2, 1)
psp.plot_BxBy_quiver(
    x, y, B1x_act, B1y_act,
    ax, title="Actual",
    x_tick_pos=XY_x_tick_pos, x_tick_labels=XY_x_tick_labels,
    y_tick_pos=XY_y_tick_pos, y_tick_labels=XY_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 2, 2)
psp.plot_BxBy_quiver(
    x, y, B1x_pred, B1y_pred,
    ax, title="Predicted",
    show_ylabel=False,
    x_tick_pos=XY_x_tick_pos, x_tick_labels=XY_x_tick_labels,
    y_tick_pos=XY_y_tick_pos, y_tick_labels=XY_y_tick_labels,
)

fig.suptitle("Final magnetic field")
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in final magnetic field magnitudes.
B1_act = np.sqrt(B1x_act**2 + B1y_act**2)
B1_pred = np.sqrt(B1x_pred**2 + B1y_pred**2)
B1_err = B1_pred - B1_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
B1_act_plot = np.flip(B1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_logarithmic_heatmap(
    B1_act_plot,
    ax, title="Actual",
    vmin=B_VMIN, vmax=B_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
B1_pred_plot = np.flip(B1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_logarithmic_heatmap(
    B1_pred_plot,
    ax, title="Predicted",
    vmin=B_VMIN, vmax=B_VMAX,
    show_ylabel=False,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
B1_err_plot = np.flip(B1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    B1_err_plot,
    ax, title="Absolute error",
    vmin=B_ABS_ERR_VMIN, vmax=B_ABS_ERR_VMAX,
    show_ylabel=False,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Final magnetic field magnitude")
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in final magnetic field divergence.
dB1x_dx_act = dBx_dx_analytical(t, x, y)
dB1y_dy_act = dBy_dy_analytical(t, x, y)
divB1_act = dB1x_dx_act + dB1y_dy_act
dB1x_dx_pred = delψ[3][-n_end:, 1]
dB1y_dy_pred = delψ[4][-n_end:, 2]
divB1_pred = dB1x_dx_pred + dB1y_dy_pred
divB1_err = divB1_pred - divB1_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
divB1_act_plot = np.flip(divB1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    divB1_act_plot,
    ax, title="Actual",
    vmin=DIV_B_VMIN, vmax=DIV_B_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
divB1_pred_plot = np.flip(divB1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    divB1_pred_plot,
    ax, title="Predicted",
    vmin=DIV_B_VMIN, vmax=DIV_B_VMAX,
    show_ylabel=False,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
divB1_err_plot = np.flip(divB1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    divB1_err_plot,
    ax, title="Absolute error",
    vmin=DIV_B_VMIN, vmax=DIV_B_VMAX,
    show_ylabel=False,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Final magnetic field divergence")
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in final number density.
n1_act = n_analytical(t, x, y)
n1_pred = ψ[0][-n_end:]
n1_err = n1_pred - n1_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
n1_act_plot = np.flip(n1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    n1_act_plot,
    ax, title="Actual",
    vmin=N_VMIN, vmax=N_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
n1_pred_plot = np.flip(n1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    n1_pred_plot,
    ax, title="Predicted",
    vmin=N_VMIN, vmax=N_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
n1_err_plot = np.flip(n1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    n1_err_plot,
    ax, title="Absolute error",
    vmin=ABS_ERR_VMIN, vmax=ABS_ERR_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Final %s" % p.dependent_variable_labels[0])
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in final pressure.
P1_act = P_analytical(t, x, y)
P1_pred = ψ[1][-n_end:]
P1_err = P1_pred - P1_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
P1_act_plot = np.flip(P1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    P1_act_plot,
    ax, title="Actual",
    vmin=P_VMIN, vmax=P_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
P1_pred_plot = np.flip(P1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    P1_pred_plot,
    ax, title="Predicted",
    vmin=P_VMIN, vmax=P_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
P1_err_plot = np.flip(P1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    P1_err_plot,
    ax, title="Absolute error",
    vmin=ABS_ERR_VMIN, vmax=ABS_ERR_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Final %s" % p.dependent_variable_labels[1])
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in x-velocity.
u1x_act = ux_analytical(t, x, y)
u1x_pred = ψ[2][-n_end:]
u1x_err = u1x_pred - u1x_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
u1x_act_plot = np.flip(u1x_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    u1x_act_plot,
    ax, title="Actual",
    vmin=U_VMIN, vmax=U_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
u1x_pred_plot = np.flip(u1x_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    u1x_pred_plot,
    ax, title="Predicted",
    vmin=U_VMIN, vmax=U_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
u1x_err_plot = np.flip(u1x_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    P1_err_plot,
    ax, title="Absolute error",
    vmin=ABS_ERR_VMIN, vmax=ABS_ERR_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Final %s" % p.dependent_variable_labels[2])
plt.show()

In [ ]:
# Plot the actual, predicted, and errors in final Bz.
B1z_act = Bz_analytical(t, x, y)
B1z_pred = ψ[5][-n_end:]
B1z_err = B1z_pred - B1z_act

# Create the figure.
fig = plt.figure(figsize=B_magnitude_figsize)

# Actual
ax = plt.subplot(1, 3, 1)
# To get the proper orientation, reshape, transpose, flip.
B1z_act_plot = np.flip(B1z_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    B1z_act_plot,
    ax, title="Actual",
    vmin=BZ_VMIN, vmax=BZ_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Predicted
ax = plt.subplot(1, 3, 2)
# To get the proper orientation, reshape, transpose, flip.
B1z_pred_plot = np.flip(B1z_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    B1z_pred_plot,
    ax, title="Predicted",
    vmin=BZ_VMIN, vmax=BZ_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

# Error
ax = plt.subplot(1, 3, 3)
B1z_err_plot = np.flip(B1z_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
psp.plot_linear_heatmap(
    B1z_err_plot,
    ax, title="Absolute error",
    vmin=BZ_VMIN, vmax=BZ_VMAX,
    show_ylabel=True,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

fig.suptitle("Final %s" % p.dependent_variable_labels[5])
plt.show()